In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet

c:\Users\Henry\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Set the default tensor type to CUDA tensors
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

    # Define the device as the first visible cuda device if available
    device = torch.device("cuda:0")
    print(f"Running on {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("Running on CPU")


Running on NVIDIA GeForce RTX 3080


c:\Users\Henry\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [3]:
# Data augmentation for the training set
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the images on the horizontal axis
    transforms.RandomRotation(10),      # Random rotation of the images by 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Randomly changing brightness, contrast, and saturation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
train_dataset_path = "C:/Users/Henry/Downloads/archive/Dataset/train"
val_dataset_path = "C:/Users/Henry/Downloads/archive/Dataset/validation"
test_dataset_path = "C:/Users/Henry/Downloads/archive/Dataset/test" 

# Data loaders
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load datasets
train_dataset = datasets.ImageFolder(train_dataset_path, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, generator=torch.Generator(device='cuda'))

val_dataset = datasets.ImageFolder(val_dataset_path, transform=train_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)

test_dataset = datasets.ImageFolder(test_dataset_path, transform=train_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8)

In [5]:
# Model
model_name = 'efficientnet-b3'
model = EfficientNet.from_pretrained(model_name)

# Adding Dropout layer
dropout_rate = 0.5  # You can adjust this value
model._dropout = nn.Dropout(p=dropout_rate)

# Modify the Final Layer for binary classification
num_classes = 2
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, num_classes)

model = model.to(device)

Loaded pretrained weights for efficientnet-b3


In [6]:
# Loss and Optimizer with L2 Regularization (Weight Decay)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) 

In [7]:
# Training Loop
num_epochs = 1
for epoch in range(num_epochs):  
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Save the model at the end of each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.item(),
    }, f"model_epoch_{epoch+1}.pth")

Epoch [1/1], Step [100/4376], Loss: 0.2489
Epoch [1/1], Step [200/4376], Loss: 0.0521
Epoch [1/1], Step [300/4376], Loss: 0.1369
Epoch [1/1], Step [400/4376], Loss: 0.0998
Epoch [1/1], Step [500/4376], Loss: 0.1661
Epoch [1/1], Step [600/4376], Loss: 0.0123
Epoch [1/1], Step [700/4376], Loss: 0.0084
Epoch [1/1], Step [800/4376], Loss: 0.0767
Epoch [1/1], Step [900/4376], Loss: 0.0717
Epoch [1/1], Step [1000/4376], Loss: 0.1912
Epoch [1/1], Step [1100/4376], Loss: 0.1155
Epoch [1/1], Step [1200/4376], Loss: 0.0249
Epoch [1/1], Step [1300/4376], Loss: 0.0849
Epoch [1/1], Step [1400/4376], Loss: 0.0734
Epoch [1/1], Step [1500/4376], Loss: 0.0665
Epoch [1/1], Step [1600/4376], Loss: 0.0830
Epoch [1/1], Step [1700/4376], Loss: 0.0277
Epoch [1/1], Step [1800/4376], Loss: 0.0093
Epoch [1/1], Step [1900/4376], Loss: 0.0339
Epoch [1/1], Step [2000/4376], Loss: 0.0654
Epoch [1/1], Step [2100/4376], Loss: 0.0184
Epoch [1/1], Step [2200/4376], Loss: 0.0880
Epoch [1/1], Step [2300/4376], Loss: 0.03

In [8]:
best_val_accuracy = 0
# Validation Loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    current_val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {100 * correct / total}%")
    # Save the model if it has the best validation accuracy so far
    if current_val_accuracy > best_val_accuracy:
        best_val_accuracy = current_val_accuracy

Validation Accuracy: 95.98508674038754%


In [9]:
# Test Loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 84.16322787712059%
